<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-23 11:19:46
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 21.85 K (0.15%)
Current PnL: -33.20 L (-20.75%)
CY Booked + Current PnL: -17.52 L (-10.95%)
-------------------
Total profit:  1.33 L
Total loss:  -34.53 L
-------------------
Total Booked + Current PnL: 9.46 L (7.11%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.1%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.82 L (70.67%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,4.24,6.09,14.59,21.57,15166.0,5970.0,103950.0,397.04,-8.31,63.0,H-SC,6.23,167.0,0.39,0.75,51.69,XR,ATH,FINANCE
18,COALINDIA,-0.85,3.27,13.94,17.67,21660.0,4922.0,155382.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.73,XY25,ATH,MINING
36,ICICIGI,1.30,3.89,14.88,19.34,30934.0,7778.0,207887.0,2252.93,-12.72,63.0,X-MC,1.94,66.0,0.25,1.50,20.44,X40,ATH,INSURANCE
77,TTKPRESTIG,-0.91,-31.17,45.42,0.09,31505.0,-31413.0,69364.0,770.00,60.94,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
51,MEDANTA,2.27,-3.51,28.59,24.08,35021.0,-4456.0,122494.0,1486.00,-4.25,54.0,X-SC,9.81,87.0,-0.13,0.88,12.19,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,4.24,6.09,14.59,21.57,15166.0,5970.0,103950.0,397.04,-8.31,63.0,H-SC,6.23,167.0,0.39,0.75,51.69,XR,ATH,FINANCE
76,TRIDENT,3.76,-29.08,81.20,28.51,53129.0,-26824.0,65430.0,48.00,-8.79,49.0,M-SC,8.46,223.0,-0.50,0.47,11.96,XR,NTT,TEXTILES
7,ATULAUTO,3.21,-15.39,72.05,45.57,120170.0,-30345.0,166787.0,844.00,3613.51,57.0,M-SC,3.91,248.0,-0.25,1.20,26.22,XY24,NTT,AUTO
28,GLAXO,2.60,6.96,30.09,39.15,64137.0,13868.0,213152.0,3466.20,-18.33,64.0,X-MC,10.89,56.0,0.22,1.54,17.24,X40,ATH,PHARMA
37,ICICIPRULI,2.30,10.26,18.57,30.73,40337.0,20209.0,217214.0,790.00,-19.31,61.0,X-MC,4.18,78.0,0.50,1.56,24.34,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,-3.04,-25.57,76.36,31.26,131156.0,-59008.0,171760.0,18931.72,-66.76,39.0,X-MC,24.74,54.0,-0.45,1.24,5.41,X40N,ATH,IT
84,WIPRO,-2.29,-15.90,104.83,72.27,149393.0,-26938.0,142510.0,420.00,-24.09,23.0,M-LC,6.94,98.0,-0.18,1.03,0.00,XR,NTT,IT
42,INFY,-2.16,-9.87,48.94,34.24,138017.0,-30876.0,282012.0,1972.00,-29.25,21.0,X-LC,5.42,3.0,-0.22,2.03,0.00,X40,NTT,IT
25,EASEMYTRIP,-2.04,-49.72,189.79,45.70,168714.0,-87920.0,88895.0,26.40,-7.61,70.0,M-SC,19.08,204.0,-0.52,0.64,46.46,XY24,NTT,TRAVEL
72,TATAELXSI,-2.04,-31.40,92.46,32.03,83621.0,-41395.0,90440.0,9161.00,-14.15,33.0,H-SC,8.59,158.0,-0.50,0.65,1.30,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.97,2.27,103.29,107.91,155970.0,3358.0,151002.0,1641.55,-8.17,59.0,M-SC,8.15,220.0,0.02,1.09,13.60,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.91,-31.17,45.42,0.09,31505.0,-31413.0,69364.0,770.00,60.94,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,-2.04,-31.40,92.46,32.03,83621.0,-41395.0,90440.0,9161.00,-14.15,33.0,H-SC,8.59,158.0,-0.50,0.65,1.30,OX40N,NTT,IT
66,SIS,-0.10,-31.90,78.08,21.26,59035.0,-35424.0,75608.0,528.00,1773.58,29.0,H-SC,4.61,168.0,-0.60,0.54,4.23,OX40N,NTT,MISC
40,INDIGOPNTS,0.74,-32.44,48.05,0.02,56662.0,-56635.0,117924.0,1408.00,79.69,28.0,M-SC,10.35,222.0,-1.00,0.85,2.57,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,-0.36,20.58,20.42,45.2,47210.0,39453.0,231195.0,1856.0,38.08,64.0,X-MC,6.69,79.0,0.84,1.67,28.77,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.85,3.27,13.94,17.67,21660.0,4922.0,155382.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.73,XY25,ATH,MINING
26,FINCABLES,1.97,2.27,103.29,107.91,155970.0,3358.0,151002.0,1641.55,-8.17,59.0,M-SC,8.15,220.0,0.02,1.09,13.60,OX40N,ATH,CABLES
50,MASFIN,4.24,6.09,14.59,21.57,15166.0,5970.0,103950.0,397.04,-8.31,63.0,H-SC,6.23,167.0,0.39,0.75,51.69,XR,ATH,FINANCE
1,ABB,-1.62,8.07,34.70,45.57,98104.0,21101.0,282720.0,7934.00,-30.51,63.0,H-MC,6.73,125.0,0.22,2.04,25.55,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,4.24,6.09,14.59,21.57,15166.0,5970.0,103950.0,397.04,-8.31,63.0,H-SC,6.23,167.0,0.39,0.75,51.69,XR,ATH,FINANCE
1,ABB,-1.62,8.07,34.70,45.57,98104.0,21101.0,282720.0,7934.00,-30.51,63.0,H-MC,6.73,125.0,0.22,2.04,25.55,AR,NTT,ELECTRICAL
26,FINCABLES,1.97,2.27,103.29,107.91,155970.0,3358.0,151002.0,1641.55,-8.17,59.0,M-SC,8.15,220.0,0.02,1.09,13.60,OX40N,ATH,CABLES
39,INDIAMART,0.86,-5.27,118.23,106.72,138132.0,-6503.0,116833.0,4810.62,-57.23,49.0,H-SC,7.95,140.0,-0.05,0.84,16.87,AR,ATH,MISC
85,ZYDUSLIFE,0.39,-4.64,42.23,35.63,84397.0,-9724.0,199850.0,1286.17,-17.12,51.0,H-MC,5.08,120.0,-0.12,1.44,11.26,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-2.16,-9.87,48.94,34.24,138017.0,-30876.0,282012.0,1972.00,-29.25,21.0,X-LC,5.42,3.0,-0.22,2.03,0.00,X40,NTT,IT
33,HCLTECH,-1.00,-7.71,34.18,23.83,76314.0,-18654.0,223270.0,1908.19,-3.23,23.0,X-LC,3.85,8.0,-0.24,1.61,7.12,X40,ATH,IT
8,AWL,-0.21,-35.79,144.83,57.19,303262.0,-116735.0,209392.0,485.00,-63.27,25.0,X-SC,17.86,81.0,-0.38,1.51,0.00,XY24,NTT,FMCG
73,TCS,-0.50,-23.40,61.32,23.58,203224.0,-101214.0,331415.0,4311.58,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.00,X40,ATH,IT
53,PGHH,-0.15,-13.95,53.40,32.01,98119.0,-29776.0,183744.0,17616.87,-41.76,32.0,X-MC,9.56,57.0,-0.30,1.32,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.63,1.96,18.44,20.77,44023.0,4592.0,238736.0,1644.00,-9.55,51.0,X-MC,0.65,64.0,0.10,1.72,9.51,X40N,NTT,BREWERIES
21,DABUR,0.23,-0.12,43.88,43.70,108718.0,-306.0,247762.0,735.00,-9.31,50.0,X-MC,1.31,70.0,-0.00,1.79,14.91,XY24,BTT,FMCG
36,ICICIGI,1.30,3.89,14.88,19.34,30934.0,7778.0,207887.0,2252.93,-12.72,63.0,X-MC,1.94,66.0,0.25,1.50,20.44,X40,ATH,INSURANCE
65,SIEMENS,1.87,-12.25,43.03,25.51,70270.0,-22790.0,163305.0,4671.50,37.11,60.0,H-LC,2.01,51.0,-0.32,1.18,20.34,AR,ATH,ELECTRICAL
75,TMPV,0.79,-15.33,57.48,33.33,158555.0,-49956.0,275844.0,600.00,-82.53,60.0,X-LC,2.68,4.0,-0.32,1.99,12.77,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.20,-24.71,106.51,55.47,195087.0,-60123.0,183163.0,181.50,-35.39,34.0,X-MC,17.62,60.0,-0.31,1.32,0.0,X40N,ATH,FINANCE
8,AWL,-0.21,-35.79,144.83,57.19,303262.0,-116735.0,209392.0,485.00,-63.27,25.0,X-SC,17.86,81.0,-0.38,1.51,0.0,XY24,NTT,FMCG
53,PGHH,-0.15,-13.95,53.40,32.01,98119.0,-29776.0,183744.0,17616.87,-41.76,32.0,X-MC,9.56,57.0,-0.30,1.32,0.0,X40,ATH,FMCG
42,INFY,-2.16,-9.87,48.94,34.24,138017.0,-30876.0,282012.0,1972.00,-29.25,21.0,X-LC,5.42,3.0,-0.22,2.03,0.0,X40,NTT,IT
73,TCS,-0.50,-23.40,61.32,23.58,203224.0,-101214.0,331415.0,4311.58,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.0,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.38,-32.11,110.39,42.84,48721.0,-20871.0,44135.0,424.00,-55.37,44.0,X-SC,17.88,80.0,-0.43,0.32,6.16,XY24,NTT,MISC
58,RELAXO,-0.42,-53.84,229.92,52.31,154076.0,-78147.0,67013.0,1176.00,-50.83,39.0,X-SC,14.07,92.0,-0.51,0.48,0.47,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.35,-47.44,157.26,35.20,106346.0,-61046.0,67624.0,2096.00,-11.34,33.0,X-SC,21.80,91.0,-0.57,0.49,0.35,X40,NTT,FOOTWEAR
51,MEDANTA,2.27,-3.51,28.59,24.08,35021.0,-4456.0,122494.0,1486.00,-4.25,54.0,X-SC,9.81,87.0,-0.13,0.88,12.19,XY24,NTT,HEALTHCARE
35,HONAUT,1.23,-23.19,86.59,43.32,108324.0,-37772.0,125100.0,58357.33,-32.62,40.0,X-SC,11.56,90.0,-0.35,0.90,1.58,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.21,-14.05,38.52,19.06,112619.0,-47801.0,292365.0,452.00,-52.69,50.0,X-LC,17.90,1.0,-0.42,2.11,5.86,X40,NTT,FMCG
73,TCS,-0.50,-23.40,61.32,23.58,203224.0,-101214.0,331415.0,4311.58,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.00,X40,ATH,IT
42,INFY,-2.16,-9.87,48.94,34.24,138017.0,-30876.0,282012.0,1972.00,-29.25,21.0,X-LC,5.42,3.0,-0.22,2.03,0.00,X40,NTT,IT
75,TMPV,0.79,-15.33,57.48,33.33,158555.0,-49956.0,275844.0,600.00,-82.53,60.0,X-LC,2.68,4.0,-0.32,1.99,12.77,XY24,NTT,AUTO
33,HCLTECH,-1.00,-7.71,34.18,23.83,76314.0,-18654.0,223270.0,1908.19,-3.23,23.0,X-LC,3.85,8.0,-0.24,1.61,7.12,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.84,-32.20,96.01,32.89,50255.0,-24864.0,52343.0,1800.00,-346.50,51.0,L-MC,11.28,258.0,-0.49,0.38,44.15,XR,NTT,BANKS
67,SONACOMS,1.04,-6.99,49.54,39.09,46612.0,-7070.0,94089.0,804.02,-26.28,65.0,M-MC,9.14,193.0,-0.15,0.68,33.10,AR,ATH,AUTO
50,MASFIN,4.24,6.09,14.59,21.57,15166.0,5970.0,103950.0,397.04,-8.31,63.0,H-SC,6.23,167.0,0.39,0.75,51.69,XR,ATH,FINANCE
18,COALINDIA,-0.85,3.27,13.94,17.67,21660.0,4922.0,155382.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.73,XY25,ATH,MINING
7,ATULAUTO,3.21,-15.39,72.05,45.57,120170.0,-30345.0,166787.0,844.00,3613.51,57.0,M-SC,3.91,248.0,-0.25,1.20,26.22,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-2.04,-49.72,189.79,45.70,168714.0,-87920.0,88895.0,26.40,-7.61,70.0,M-SC,19.08,204.0,-0.52,0.64,46.46,XY24,NTT,TRAVEL
7,ATULAUTO,3.21,-15.39,72.05,45.57,120170.0,-30345.0,166787.0,844.00,3613.51,57.0,M-SC,3.91,248.0,-0.25,1.20,26.22,XY24,NTT,AUTO
1,ABB,-1.62,8.07,34.70,45.57,98104.0,21101.0,282720.0,7934.00,-30.51,63.0,H-MC,6.73,125.0,0.22,2.04,25.55,AR,NTT,ELECTRICAL
67,SONACOMS,1.04,-6.99,49.54,39.09,46612.0,-7070.0,94089.0,804.02,-26.28,65.0,M-MC,9.14,193.0,-0.15,0.68,33.10,AR,ATH,AUTO
82,VOLTAS,-0.36,20.58,20.42,45.20,47210.0,39453.0,231195.0,1856.00,38.08,64.0,X-MC,6.69,79.0,0.84,1.67,28.77,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.69
1,25,45.46
2,50,77.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.16
MC    30.12
LC    24.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.00
X40      24.02
X40N     14.78
AR        8.99
XY25      8.79
XR        8.52
OX40N     7.10
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.68
H-SC    22.14
X-LC    21.43
X-SC    11.56
M-SC    10.71
H-MC     4.99
M-MC     2.07
H-LC     1.18
L-LC     1.12
M-LC     1.03
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.63,-13.90,52.43
FINANCE,13.06,-15.14,60.45
IT,11.47,-37.75,106.78
MISC,6.87,-36.16,91.85
ELECTRICAL,6.25,-8.29,47.80
PAINTS,5.02,-32.49,52.17
INSURANCE,4.86,-0.47,34.86
PHARMA,4.13,-2.75,35.10
AUTO,3.87,-16.28,60.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3381268.0,21
X40,1387882.0,15
AR,1325353.0,10
XR,1308265.0,12
X40N,1071117.0,10
OX40N,805930.0,10
XY25,399213.0,6
SR,302563.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3718631.0,24
M-SC,1431091.0,14
X-MC,1367924.0,15
X-LC,1300368.0,12
X-SC,1199958.0,10
H-MC,386896.0,3
L-SC,186779.0,2
M-LC,149393.0,1
M-MC,98366.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1268281.0      6
           AR         951316.0      5
           XR         840796.0      7
M-SC       XY24       825140.0      6
X-LC       X40        674492.0      6
X-SC       XY24       615807.0      4
X-MC       X40        534691.0      7
           X40N       435725.0      4
X-SC       X40N       405452.0      4
H-SC       OX40N      355675.0      4
M-SC       OX40N      354560.0      5
H-SC       SR         302563.0      2
X-LC       XY24       270012.0      2
           X40N       229940.0      2
H-MC       XY24       204395.0      1
X-MC       XY25       199875.0      2
           XY24       197633.0      2
H-MC       AR         182501.0      2
X-SC       X40        178699.0      2
M-SC       XR         176737.0      2
M-LC       XR         149393.0      1
X-LC       XY25       125924.0      2
L-SC       OX40N       95695.0      1
           XR          91084.0      1
M-SC       AR          74654.0      1
H-LC       AR          70270.0      1
M-MC       XY25        51754.0      1
L-MC       XR          50255.0      1
M-MC       AR          46612.0      1
L-LC       XY25        21660.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
